Geokodowanie ogloszen sprzedarzy w Krakowie, ogloszenia z  okresu 2019-03 : 2019-12

dane sciagniete skryptem:

https://ofedoruk.pl/szukamy-mieszkania-za-pomoca-pythona-i-beautiful-soup/,

In [1]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")
import numpy as np 
import pandas as pd 
import seaborn as sns
import datetime
import time 
import geopandas as gpd
import difflib
from difflib import SequenceMatcher

In [10]:
# wczytanie ogloszen
df = pd.read_csv('offers.csv', header = 0, sep = ";", error_bad_lines=False)
il_ogloszen = df.shape[0]
print('mamy ', il_ogloszen, ' ogloszen')

In [3]:
# pobranie listy ulic z warstwy OpenStreetMap
flats = gpd.read_file(r'shp\shp\drogi_krk_osm_dis.shp')

lista_nazw = []

for i in flats.itertuples():
    ulica = i.name
    ulica_list = ulica.split(" ")
    lista_nazw.append(ulica_list[-1])

# sprawdzenie po kolei wyrazow z opisu czy sie zgadzaja   
def szukaj_ulicy(cell):
    opis_list = cell.split(" ")
    for slowo in opis_list:
        if len(slowo) > 4:
            for ulica in lista_nazw:
                ratio = SequenceMatcher(None, slowo, ulica).ratio()
                if ratio > 0.8:
                    return ulica

In [4]:
def czyszczenie(df, ALL_NUMERIC_COLUMNS):
    # numerical data cleaning
    
    df = df.rename(columns={'pokoje': 'no_rooms', 'metraz': 'flat_area',
                             'cena_za_metr': 'price'})
    df[ALL_NUMERIC_COLUMNS]= df[ALL_NUMERIC_COLUMNS].replace(['>',' ', '/', 'pokoje',
                 'pokój', 'pokoi', 'zł', 'm²'],'', regex=True)
    df[ALL_NUMERIC_COLUMNS]= df[ALL_NUMERIC_COLUMNS].replace(',','.', regex=True)
    df[ALL_NUMERIC_COLUMNS] = df[ALL_NUMERIC_COLUMNS].astype(float)
    
    # outliery z common sense
    df = df.loc[df['price'] < 15000]
    df = df.loc[df['price'] > 3000]
    df = df.drop_duplicates(ALL_NUMERIC_COLUMNS, keep='first')  #usuwanie duplikatow

    return df

In [5]:
# text data cleaning 
lista_opisow = ['opis', 'dzielnica']
lista_zbednych  = ['/', 'm²', ":", "-", '.']


df['lokator_dzielnica'] = "Krakow" + " , " + df['dzielnica']
ALL_NUMERIC_COLUMNS= ['flat_area', 'no_rooms',  'price', ]

df = czyszczenie(df, ALL_NUMERIC_COLUMNS)
# df['opis'] = df['opis'].replace(lista_zbednych,'', regex=True)

df_probe = df.iloc[:100]
# przypisanie ulicy
df_probe['opis'] = df_probe['opis'].replace(["ul.", "-", ",", "os."]," ", regex=True)
df_probe['ulica'] = df_probe['opis'].apply(szukaj_ulicy)
df_probe['lokator_ulica'] = "Kraków " +  df_probe['ulica']

display(df_probe.head(50))

,opis,dzielnica,no_rooms,cena,flat_area,price,link,data,lokator_dzielnica,ulica,lokator_ulica
0,w sąsiedztwie Rynku Głównego / 2 miejsca p ojo...,: Stare Miasto,4.0,777 000,87.20,8911.0,https://www.otodom.pl/oferta/w-sasiedztwie-ryn...,2019-03-07 15:26:00.407141,"Krakow , : Stare Miasto",None,NaN
1,Apartament u stóp Wawelu,: Stare Miasto,2.0,745 000,52.50,14190.0,https://www.otodom.pl/oferta/apartament-u-stop...,2019-03-07 15:26:00.407141,"Krakow , : Stare Miasto",Wawelu,Kraków Wawelu
2,Mieszkanie 57 Kraków,: Ruczaj,3.0,430 000,57.00,7544.0,https://www.otodom.pl/oferta/mieszkanie-57-m-k...,2019-03-07 15:26:00.407141,"Krakow , : Ruczaj",None,NaN
3,Mieszkanie 48 Kraków,: Bronowice,2.0,399 500,48.00,8323.0,https://www.otodom.pl/oferta/mieszkanie-48-m-k...,2019-03-07 15:26:00.407141,"Krakow , : Bronowice",None,NaN
4,Mieszkanie 52 Kraków,: Krowodrza,3.0,388 000,52.00,7462.0,https://www.otodom.pl/oferta/mieszkanie-52-m-k...,2019-03-07 15:26:00.407141,"Krakow , : Krowodrza",None,NaN
5,Kawalerka Kraków Podgórze,: Podgórze,1.0,180 000,17.60,10227.0,https://www.otodom.pl/oferta/kawalerka-krakow-...,2019-03-07 15:26:00.408144,"Krakow , : Podgórze",None,NaN
6,2/3 pokoje WADOWICKA Aktualne,: Podgórze,3.0,399 000,45.00,8867.0,https://www.otodom.pl/oferta/2-3-pokoje-wadowi...,2019-03-07 15:26:00.408144,"Krakow , : Podgórze",None,NaN
7,Do remontu w niskiej zabudowie + ogródek!,: Stare Dębniki,2.0,286 000,35.60,8034.0,https://www.otodom.pl/oferta/do-remontu-w-nisk...,2019-03-07 15:26:00.408144,"Krakow , : Stare Dębniki",budowie,Kraków budowie
8,4pok.w pobliżu terenów zielonych Prywatny Ogró...,: Czyżyny,4.0,446 300,75.00,5951.0,https://www.otodom.pl/oferta/4pok-w-poblizu-te...,2019-03-07 15:26:00.408144,"Krakow , : Czyżyny",None,NaN
9,3pok.z dwoma balkonami w zielonej okolicy Czyż...,: Czyżyny,3.0,378 752,59.18,6400.0,https://www.otodom.pl/oferta/3pok-z-dwoma-balk...,2019-03-07 15:26:00.408144,"Krakow , : Czyżyny",None,NaN


In [6]:
def geokoduj(df, pole, rodzaj):
    df['geo_location'] = df[pole].apply(geocode)
    df['rodzaj'] = rodzaj
    located = df.loc[df["geo_location"].notnull()]
    not_located = df.loc[df["geo_location"].isnull()]
    return(located, not_located)

In [8]:
# geocoding
import geopy
import geopandas
from geopy.geocoders import Nominatim

locator = Nominatim(user_agent ='nlp2')
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

# geokoduj1 = geokoduj(df_probe, 'lokator_all', 'opis')
geokoduj2 = geokoduj(df_probe.loc[df_probe["ulica"].notnull()], 'lokator_ulica', 'ulica')
geokoduj3 = geokoduj(df_probe.loc[df_probe["ulica"].isnull()], 'lokator_dzielnica',' dzielnica')

#located_opis = geokoduj1[0]
located_ulice = geokoduj2[0]
located_dzielnice = geokoduj3[0]

# merging of diffrent types of geocodings
located = pd.concat([located_ulice, located_dzielnice])

il_ogloszen = located.shape[0]
display('mamy ', il_ogloszen, ' zgeokodowanych ogloszen z testowych 100')

# 3 - create longitude, laatitude and altitude from location column (returns tuple)
located['point'] = located['geo_location'].apply(lambda loc: tuple(loc.point) if loc else None)

# 4 - split point column into latitude, longitude and altitude columns
located[['latitude', 'longitude', 'altitude']] = pd.DataFrame(located['point'].tolist(), index=located.index)

located.to_csv('geokoding_20200107.csv')

'mamy '

99

' zgeokodowanych ogloszen z testowych 100'

In [9]:
# wyswietlenie punktow na mapie, razem z pop ups
import folium

Mmap = folium.Map(location=[50.05, 19.95], zoom_start=12, )

def getColor(rodzaj):
    if rodzaj == 'opis':
        return 'blue'
    elif rodzaj == 'ulica':
        return 'red'
    else:
        return 'blue'

for lat, lon, opis, link, dzielnica, rodzaj, ulica in zip(located['latitude'], located['longitude'], located['opis'],
                                                          located['link'],  located['dzielnica'],  located['rodzaj'], 
                                                          located['ulica']):
        folium.CircleMarker(
            [lat, lon],
            radius=.15,
            popup = (str(link) +'<br>''<br>' + "zgeokodowano na podstawie : " + str(rodzaj)),
            color= getColor(rodzaj),
            key_on = ulica,
            threshold_scale=[0,1,2,3],
            fill_color = getColor(rodzaj),
            fill=True,
            fill_opacity=0.7
            ).add_to(Mmap)

Mmap